In [254]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [251]:
# https://movie.douban.com/top250

In [294]:
rows = []

for num in range(0, 226, 25):
    url = f'https://movie.douban.com/top250?start={num}&filter='
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    films = doc.find(class_ = 'grid_view').find_all('li')
    print('scraping', url)
    for film in films:
        row = {}
        row['cn_title'] = film.find(class_ = 'hd').span.text.strip()
        row['rank'] = film.em.text
        row['eng_title'] = film.find(class_ = 'hd').span.find_next_sibling('span').text.strip().strip(' / ').strip('\xa0')
        row['more_title'] = film.find(class_ = 'hd').find('span', class_ = 'other').text.strip().strip(' / ').strip('\xa0')
        director = re.findall(r'[^\u4e00-\u9fff]+', film.find(class_ = 'bd').text)
        try:
            row['eng_dir'] = re.findall(r'^(.+)\xa0\xa0\xa0.+' , ''.join(director).strip().strip(': · '))[0]
        except:
            pass
        try:
            row['country'] = re.findall(r'\d\d\d\d\s/\s([\u4e00-\u9fff]{2})', film.find(class_ = 'bd').text.strip())[0]
        except:
            pass
        row['year'] = re.findall(r'\d\d\d\d' ,film.find(class_ = 'bd').p.text.strip())[0]
        row['score'] = film.find(class_ = 'rating_num').text
        row['num_of_reviews'] = re.findall(r'\d+',film.find(class_ = 'star').find('span', content = False, class_ = False).text.strip())[0]
        row['url'] = film.find(class_ = 'info').a['href']
        row['img'] = film.find(class_ = 'pic').img['src']
        rows.append(row)

rows
    

scraping https://movie.douban.com/top250?start=0&filter=
scraping https://movie.douban.com/top250?start=25&filter=
scraping https://movie.douban.com/top250?start=50&filter=
scraping https://movie.douban.com/top250?start=75&filter=
scraping https://movie.douban.com/top250?start=100&filter=
scraping https://movie.douban.com/top250?start=125&filter=
scraping https://movie.douban.com/top250?start=150&filter=
scraping https://movie.douban.com/top250?start=175&filter=
scraping https://movie.douban.com/top250?start=200&filter=
scraping https://movie.douban.com/top250?start=225&filter=


[{'cn_title': '肖申克的救赎',
  'rank': '1',
  'eng_title': 'The Shawshank Redemption',
  'more_title': '月黑高飞(港)  /  刺激1995(台)',
  'eng_dir': 'Frank Darabont',
  'country': '美国',
  'year': '1994',
  'score': '9.6',
  'num_of_reviews': '1468443',
  'url': 'https://movie.douban.com/subject/1292052/',
  'img': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p480747492.jpg'},
 {'cn_title': '霸王别姬',
  'rank': '2',
  'eng_title': '再见，我的妾  /  Farewell My Concubine',
  'more_title': '再见，我的妾  /  Farewell My Concubine',
  'eng_dir': 'Kaige Chen',
  'country': '中国',
  'year': '1993',
  'score': '9.6',
  'num_of_reviews': '1087882',
  'url': 'https://movie.douban.com/subject/1291546/',
  'img': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p1910813120.jpg'},
 {'cn_title': '这个杀手不太冷',
  'rank': '3',
  'eng_title': 'Léon',
  'more_title': '杀手莱昂  /  终极追杀令(台)',
  'eng_dir': 'Luc Besson',
  'country': '法国',
  'year': '1994',
  'score': '9.4',
  'num_of_reviews': '1336217',
  'url': 'h

In [307]:
df = pd.DataFrame(rows)

In [308]:
df.head()

,cn_title,country,eng_dir,eng_title,img,more_title,num_of_reviews,rank,score,url,year
0,肖申克的救赎,美国,Frank Darabont,The Shawshank Redemption,https://img3.doubanio.com/view/photo/s_ratio_p...,月黑高飞(港) / 刺激1995(台),1468443,1,9.6,https://movie.douban.com/subject/1292052/,1994
1,霸王别姬,中国,Kaige Chen,再见，我的妾 / Farewell My Concubine,https://img3.doubanio.com/view/photo/s_ratio_p...,再见，我的妾 / Farewell My Concubine,1087882,2,9.6,https://movie.douban.com/subject/1291546/,1993
2,这个杀手不太冷,法国,Luc Besson,Léon,https://img3.doubanio.com/view/photo/s_ratio_p...,杀手莱昂 / 终极追杀令(台),1336217,3,9.4,https://movie.douban.com/subject/1295644/,1994
3,阿甘正传,美国,Robert Zemeckis,Forrest Gump,https://img3.doubanio.com/view/photo/s_ratio_p...,福雷斯特·冈普,1154031,4,9.4,https://movie.douban.com/subject/1292720/,1994
4,美丽人生,意大,Roberto Benigni,La vita è bella,https://img3.doubanio.com/view/photo/s_ratio_p...,一个快乐的传说(港) / Life Is Beautiful,676762,5,9.5,https://movie.douban.com/subject/1292063/,1997


In [311]:
df.country.value_counts()

U.S.           120
Japan           31
Hong Kong       18
U.K.            14
China           13
Korea           10
France           8
Italy            8
Germany          5
India            4
Taiwan           4
Iran             2
Australia        2
Lebanon          1
Thai             1
Brazil           1
Sweden           1
Ireland          1
Argentina        1
Danmark          1
New Zealand      1
Spain            1
Name: country, dtype: int64

In [310]:
df = df.replace({'country': {
    '美国': 'U.S.',
    '中国': 'China',
    '韩国': 'Korea',
    '香港': 'Hong Kong',
    '日本': 'Japan',
    '英国': 'U.K.',
    '意大': 'Italy',
    '法国': 'France',
    '德国': 'Germany',
    '印度': 'India',
    '台湾': 'Taiwan',
    '澳大': 'Australia',
    '伊朗': 'Iran',
    '丹麦': 'Danmark',
    '黎巴': 'Lebanon',
    '巴西': 'Brazil',
    '瑞典': 'Sweden',
    '阿根': 'Argentina',
    '新西': 'New Zealand',
    '泰国': 'Thai',
    '爱尔': 'Ireland',
    '西班': 'Spain'
}})

In [312]:
df = df[['rank', 'cn_title', 'eng_title', 'more_title', 'country', 'eng_dir', 'score', 'num_of_reviews', 'year', 'img', 'url']]

In [313]:
df.to_csv('Douban_Top250.csv', index = False)